In [1]:
import sys
import os
import boto3
import json
import rasterio

import GOSTrocks.rasterMisc as rMisc
import geopandas as gpd
import pandas as pd
import xml.etree.ElementTree as ET

from tqdm.notebook import tqdm

sys.path.insert(0, "../../src")
import GOSTrocks.dataMisc as dMisc
import GOSTrocks.mapMisc as mapMisc


import warnings
from urllib3.exceptions import InsecureRequestWarning

# Suppress only the InsecureRequestWarning from urllib3
warnings.simplefilter('ignore', InsecureRequestWarning)

%load_ext autoreload
%autoreload 2

# Update to version 3.1
In the summer of 2025, we received a complete update of the Fathom dataset labelled 3.1. This notebook will generate new VRTs for version 3.1 by manually editing the vrts from version 3.0

In [2]:
# Get a list of all folders in the new fathom 3.1 bucket
s3_bucket = "wbg-geography01"
s3_prefix = "FATHOM/v31/"
s3 = boto3.client('s3', verify=False)

paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=s3_bucket, Prefix=s3_prefix, Delimiter='/')
new_fathom_models = []
for page in pages:
    for prefix in page.get('CommonPrefixes', []):
        new_fathom_models.append(prefix.get('Prefix'))

new_fathom_models = [f.split("/")[-2] for f in new_fathom_models if f.endswith('v3.1/')]
new_fathom_models[:5]

['FLOOD_MAP-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.1',
 'FLOOD_MAP-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP1_2.6-PERCENTILE50-v3.1',
 'FLOOD_MAP-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP2_4.5-PERCENTILE50-v3.1',
 'FLOOD_MAP-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.1',
 'FLOOD_MAP-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP5_8.5-PERCENTILE50-v3.1']

In [3]:
# Get a list of vrts to edit
s3_prefix = "FATHOM/"
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=s3_bucket, Prefix=s3_prefix, Delimiter='/')
vrt_files = []
for page in pages:    
    for obj in page.get('Contents', []):
        key = obj['Key']
        if key.endswith('.vrt'):
            vrt_files.append(key)

vrt_files = [f.split("/")[-1] for f in vrt_files]
vrt_files[:5]

['FLOOD_MAP-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.1.vrt',
 'FLOOD_MAP-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP1_2.6-PERCENTILE50-v3.1.vrt',
 'FLOOD_MAP-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP2_4.5-PERCENTILE50-v3.1.vrt',
 'FLOOD_MAP-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.1.vrt',
 'FLOOD_MAP-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP5_8.5-PERCENTILE50-v3.1.vrt']

In [ ]:
# for each new_fathom_model, find the corresponding vrt and edit it
''' This isn't working ...
for model in new_fathom_models:
    model_base = "-".join(model.split("-")[1:-1]) 
    matching_vrts = [v for v in vrt_files if model_base in v]
    if len(matching_vrts) == 1:
        cur_vrt = matching_vrts[0]
        s3_object = s3.get_object(Bucket=s3_bucket, Key=f"FATHOM/{cur_vrt}")
    
        # Read the content of the object and decode it (assuming UTF-8 encoding)
        xml_data = s3_object['Body'].read().decode('utf-8')

    break
'''

In [ ]:
class generate_vrt_from_template:
    def __init__(self, template_vrt, new_dataset, old_dataset=None, new_vrt=None):
        self.template_vrt = template_vrt
        self.new_dataset = new_dataset
        self.old_dataset = os.path.basename(template_vrt)[:-4] if old_dataset is None else old_dataset
        self.new_vrt = self.template_vrt.replace(self.old_dataset, self.new_dataset) if new_vrt is None else new_vrt
    def update_vrt(self):
        tree = ET.parse(self.template_vrt)
        root = tree.getroot()

        for child in root.iter("SourceFilename"):
            cur_tif = child.text.split("/")[-1]
            child.text = self.new_dataset + "/" + cur_tif

        tree.write(self.new_vrt, xml_declaration=False)

# For each of the new Fathom models, create a new VRT file, based on existing templates
out_folder = "C:/Temp/fathom_vrts_v31/"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

standard_template_vrt = os.path.abspath("../sample_data/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2080-SSP5_8.5-PERCENTILE50-v3.0.vrt")
coastal_template_vrt = os.path.abspath("../sample_data/GLOBAL-1ARCSEC-NW_OFFSET-1in500-COASTAL-UNDEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt")


for c_model in tqdm(new_fathom_models):
    cur_template = standard_template_vrt
    if "COASTAL" in c_model:
        cur_template = coastal_template_vrt 
    out_vrt = os.path.join(out_folder, f"{c_model}.vrt")
    old_dataset = 'v2023/' + c_model.replace("v3.1", "v3.0").replace("FLOOD_MAP", "GLOBAL")
    new_dataset = 'v31/' + c_model
    generate_vrt_from_template(cur_template, new_dataset, old_dataset=old_dataset, new_vrt=out_vrt).update_vrt()
    s3.upload_file(out_vrt, s3_bucket, f"FATHOM/{c_model}.vrt")

In [ ]:
# Generate a table of all new VRT files
vrt_table = []
for c_model in new_fathom_models:
    vrt_path = f"s3://{s3_bucket}/FATHOM/{c_model}.vrt"
    c = c_model.split("-")
    cur_vals = [c_model, c[3], c[4], c[5], c[7], c[8], vrt_path]
    vrt_table.append(cur_vals)
    

# Write out the table to S3
storage_options = {
    'client_kwargs': {
        'verify': False # Disable SSL verification
    }
}
out_s3_dataframe = f's3://{s3_bucket}/FATHOM/fathom_v31_vrt_table.csv'
pd.DataFrame(vrt_table, 
             columns=["model_name", "return", "type", "defended", "year", "scenario", "vrt_path"]).to_csv(
                out_s3_dataframe, 
                storage_options=storage_options)

In [ ]:
# Test results
iso3 = 'KEN'
world_filepath = r'C:\WBG\Work\data\ADMIN\NEW_WB_BOUNDS\FOR_PUBLICATION\gpkg\WB_GAD_ADM0_complete.gpkg'
world = gpd.read_file(world_filepath)
inB = world.loc[world["ISO_A3"] == iso3].copy()

temp_s3_path = f"s3://{s3_bucket}/FATHOM/{c_model}.vrt"

# open the VRT with rasterio and clip to the country boundary
with rasterio.Env(GDAL_HTTP_UNSAFESSL="YES"):
    with rasterio.open(temp_s3_path) as src:
        out_image, out_transform = rMisc.clipRaster(src, inB, crop=True)

In [ ]:
with rMisc.create_rasterio_inmemory(out_transform, out_image) as clipped_raster:
    mapMisc.static_map_raster(clipped_raster)

In [ ]:
rMisc.clipRaster?

In [ ]:
s3_bucket = "wbg-geography01"
s3_prefix = "FATHOM/v2023/"
s3_out = os.path.join("s3://", s3_bucket, s3_prefix)

s3 = boto3.resource("s3")
my_bucket = s3.Bucket(s3_bucket)

# Find all files already copied
all_folders = []
for obj in my_bucket.objects.filter(Prefix=s3_prefix):
    all_folders.append(obj.key.split("/")[-2])
all_folders
all_folders = list(set(all_folders))

In [ ]:
# build list of rasters for generating VRT
local_path = os.path.join(
    "v2023",
    "GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0",
)
in_folder = os.path.join(template_folder, local_path)  # noqa
all_tiffs = [f"{local_path}/{x}" for x in os.listdir(in_folder)]

with open(os.path.join(template_folder, "s3_tiffs.txt"), "w") as out:  # noqa
    for p in all_tiffs:
        out.write(f"{p}\n")

In [ ]:
template_folder = "/home/wb411133/temp"
coastal_template = os.path.join(
    template_folder,
    "GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0.0.vrt",
)
other_template = os.path.join(
    template_folder,
    "GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt",
)


class generate_vrt_from_template:
    def __init__(self, template_vrt, new_dataset):
        self.template_vrt = template_vrt
        self.new_dataset = new_dataset
        self.old_dataset = os.path.basename(template_vrt)[:-4]
        self.new_vrt = self.template_vrt.replace(self.old_dataset, self.new_dataset)

    def update_vrt(self):
        tree = ET.parse(self.template_vrt)
        root = tree.getroot()

        for child in root.iter("SourceFilename"):
            child.text = child.text.replace(self.old_dataset, self.new_dataset)

        tree.write(self.new_vrt, xml_declaration=False)


"""
new_ds = "GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0"
xx = generate_vrt_from_template(template_vrt, new_ds)
xx.update_vrt()
"""

In [ ]:
# Create template vrts
for new_ds in all_folders:
    flood_type = new_ds.split("-")[4]
    vrt_template = other_template
    if flood_type == "COASTAL":
        vrt_template = coastal_template
    xx = generate_vrt_from_template(vrt_template, new_ds)
    xx.update_vrt()

In [ ]:
# aws s3 cp . s3://wbg-geography01/FATHOM/ --exclude "*" --include "*.vrt"  --recursive

# Inspect copied VRTs

In [ ]:
dMisc.get_fathom_vrts()

In [ ]:
s3_bucket = "wbg-geography01"
s3_prefix = "FATHOM/"

s3 = boto3.resource("s3")
my_bucket = s3.Bucket(s3_bucket)

all_vrts = []
for o in my_bucket.objects.filter(Prefix=s3_prefix):
    if o.key.endswith(".vrt"):
        print(o.key)
        full_vrt_path = f"s3://{s3_bucket}/{o.key}"
        all_vrts.append(full_vrt_path)

In [ ]:
all_vrts.head()

In [ ]:
all_vrts["FLOOD_TYPE"].value_counts()

In [ ]:
all_vrts = dMisc.get_fathom_vrts(True)
all_res = {}
for idx, row in all_vrts.iterrows():
    # vrt_path = row['PATH']
    # xx = rasterio.open(vrt_path)
    filename = os.path.basename(row["PATH"])
    year = row["YEAR"]
    climate_model = row["CLIMATE_MODEL"]
    if climate_model == "PERCENTILE50":
        climate_model = "CURRENT"
    flood_type = row["FLOOD_TYPE"].lower()
    defence = row["DEFENCE"].lower()
    ret = row["RETURN"]
    label = "_".join([flood_type, defence, ret, climate_model, year])
    ret = ret.replace("in", " in ")
    if year == "2020":
        description = f"Global {defence} {flood_type} flood model based on current climate. Flood depth is measured in cm expected flood depth, based on a {ret} year return period."
    else:
        description = f"Global {defence} {flood_type} flood model based on {climate_model} climate model for year {year}. Flood depth is measured in cm expected flood depth, based on a {ret} year return period."
    all_res[label] = {"description": description, "filename": row["PATH"]}

In [ ]:
all_res

In [ ]:
with open("fathom_file_descriptions.json", "w") as out_f:
    json.dump(all_res, out_f)

In [ ]:
json.dump?

# DEBURGGING

In [ ]:
# Generate a list of files for gdalbuildvrt
folder = "/home/wb411133/temp/v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0"
all_files = [
    f"v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/{x}"
    for x in os.listdir(folder)
]
all_files